<a href="https://colab.research.google.com/github/yakutaoleg/DataScience/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
import pandas as pd
from datetime import timedelta, datetime

path = 'drive/My Drive/Colab Notebooks/Vizor/Data/'
filename_ads = 'ads_data'

df_ads =  pd.read_csv(path + filename_ads + '.csv')

df_ads.head()

,date,campaign_id,ad_set_id,ad_id,cpm,ctr,spend,clicks,impressions,breakdown_set,breakdowns,platform
0,2021-01-01,23847157133130369,23847157133120369,23847157133270369,11.592920,0.000000,3.93,0,339,{country},"{""country"": ""UK""}",ios
1,2021-01-01,23847157133170369,23847157133180369,23847157133260369,8.307427,0.518135,4.81,3,579,{country},"{""country"": ""UK""}",android
2,2021-01-01,23847157133130369,23847157133120369,23847157133250369,23.848580,0.630915,7.56,2,317,{country},"{""country"": ""UK""}",ios
3,2021-01-01,23847157133130369,23847157133120369,23847157133240369,17.602996,0.374532,4.70,1,267,{country},"{""country"": ""UK""}",ios
4,2021-01-01,23847157133130369,23847157133120369,23847157133230369,23.512397,0.000000,5.69,0,242,{country},"{""country"": ""UK""}",ios


In [127]:
filename_user = 'user_data'

df_user =  pd.read_csv(path + filename_user + '.csv')
# заменяю все значения NaN на 0 в колонке доходи в USD, для дальнейших расчетов
df_user['event_revenue_usd'].fillna(0, inplace=True)

df_user.head()

,attributed_touch_time,install_time,event_time,event_name,event_revenue_usd,media_source,channel,campaign_id,adset_id,ad_id,country_code,user_id
0,NaN,2021-01-01 00:08:10.721+00,2021-01-01 00:08:10.721+00,install,0.0,NaN,NaN,NaN,NaN,NaN,VN,1609459686080-7483030925743023347
1,NaN,2021-01-01 00:39:23.948+00,2021-01-01 00:39:23.948+00,install,0.0,NaN,NaN,NaN,NaN,NaN,US,1609461562630-105981105396801528
2,NaN,2021-01-01 00:40:36.525+00,2021-01-01 00:40:36.525+00,reinstall,0.0,NaN,NaN,NaN,NaN,NaN,CZ,1609461625904-7673362380866907786
3,NaN,2021-01-01 00:49:03.486+00,2021-01-01 00:49:03.486+00,reinstall,0.0,NaN,NaN,NaN,NaN,NaN,PT,1609462209340-904240965619919015
4,NaN,2021-01-01 00:51:53.395+00,2021-01-01 00:51:53.395+00,install,0.0,NaN,NaN,NaN,NaN,NaN,US,1609279214347-3174174


In [128]:
# 3.	Рассчитайте базовые метрики для когорты 8-14.01.2021:
start_date_string = '2021-01-08'
start_date = datetime.strptime(start_date_string, '%Y-%m-%d')
end_date_string = '2021-01-15'
end_date = datetime.strptime(end_date_string, '%Y-%m-%d')
# создаю срезы по когорте.
df_ads_8_14 = df_ads.loc[(df_ads['date'] >= start_date_string) & (df_ads['date'] < end_date_string)]
df_user_8_14 = df_user.loc[(df_user['event_time'] >= start_date_string) & (df_user['event_time'] < end_date_string)]

In [131]:
cost = df_ads_8_14['spend'].sum()
print('Cost = ',cost)

impressions = df_ads_8_14['impressions'].sum() 
print('Impressions = ',impressions)

installs = df_user_8_14['event_name'].value_counts()['install']
print('Installs = ',installs)

# выбираю уникальных пользователей, которые производили оплату на 3 день когорты
start_3_day = start_date + timedelta(days = 2)
end_3_day = start_date + timedelta(days = 3)
df_slice_3_day = df_user_8_14.loc[(df_user_8_14['event_time'] >= start_3_day.strftime('%Y-%m-%d')) 
                              & (df_user_8_14['event_time'] < end_3_day.strftime('%Y-%m-%d'))]

df_buyers_3 = df_slice_3_day.loc[df_slice_3_day['event_revenue_usd'] != 0]
df_buyers_3 = df_buyers_3.drop_duplicates(subset=['user_id'])

buyers_3 = df_buyers_3['user_id'].count()
print('Buyers 3 = ',buyers_3)

# выбираю уникальных пользователей, которые производили оплату за весь срез когорты
df_user_8_14_revenue = df_user_8_14.loc[df_user_8_14['event_revenue_usd'] != 0]
df_user_8_14_revenue = df_user_8_14_revenue.drop_duplicates(subset=['user_id'])

buyers = df_user_8_14_revenue['user_id'].count()
print('Buyers = ',buyers)

# CPM (Стоимость размещения рекламы/Количество показов объявления) * 1000
CPM = cost / impressions * 1000
print('CPM = ',CPM)

# IPM число установок на 1000 показов
IPM = installs / impressions * 1000
print('IPM = ',IPM)

# CPI – Затраты на рекламу / Количество установок.
CPI = cost / installs
print('CPI = ',CPI)

# Накопительный ARPU за 7 дней = Доход (выбранной когорты за 7 дней) / Число пользователей (в рамках выбранной когорты)
end_7_day = start_date + timedelta(days = 7)
df_slice_7_days = df_user_8_14.loc[df_user_8_14['event_time'] < end_7_day.strftime('%Y-%m-%d')]
df_slice_7_days = df_slice_7_day.drop_duplicates(subset=['user_id'])
count_users_7_days = df_slice_7_day['user_id'].count()
revenue_7_days = df_slice_7_days['event_revenue_usd'].sum()


ARPU_7 = revenue_7_days / count_users_7_days
print('ARPU_7 = ',ARPU_7)

# Накопительный ARPPU за 7 дней = Доход (выбранной когорты за 7 дней) / Число плативших пользователей (в рамках выбранной когорты)
df_pay_users_7_days = df_slice_7_days.loc[df_slice_7_days['event_revenue_usd'] != 0]

df_pay_users_7_days = df_pay_users_7_days.drop_duplicates(subset=['user_id'])
count_pay_users_7_days = df_pay_users_7_days['user_id'].count()


ARPPU_7 = revenue_7_days / count_pay_users_7_days
print('ARPPU_7 = ',ARPPU_7)

# Накопительный ARPPU = Доход / Число пользователей
revenue_8_14 = df_user_8_14['event_revenue_usd'].sum()
df_user_8_14_users = df_user_8_14.drop_duplicates(subset=['user_id'])
users_uniq_count = df_user_8_14_users['user_id'].count()

ARPU = revenue_8_14 / users_uniq_count
print('ARPPU_7 = ',ARPPU_7)

# Cost per buyer - цена за плательщика на указанный день когорты. Стоимость размещения рекламы (накопительный) / число плательщиков (накопительный)
def cost_per_buyer(df_ads, df_user, start_date, n_day):   
    
    end_n_day = start_date + timedelta(days = n_day)

    df_buyer_n = df_user.loc[df_user['event_time'] < end_n_day.strftime('%Y-%m-%d')]
    df_buyer_n = df_buyer_n.loc[df_buyer_n['event_revenue_usd'] != 0]
    df_buyer_n = df_buyer_n.drop_duplicates(subset=['user_id'])  
    count_buyers_n = df_buyer_n['user_id'].count()

    df_ads_n = df_ads.loc[df_ads['date']< end_n_day.strftime('%Y-%m-%d')]
    cost_n = df_ads_n['spend'].sum()
    
    result_value = cost_n / count_buyers_n
    return result_value

CPB_1 = cost_per_buyer(df_ads_8_14, df_user_8_14, start_date, 1)
CPB_3 = cost_per_buyer(df_ads_8_14, df_user_8_14, start_date, 3)
CPB_5 = cost_per_buyer(df_ads_8_14, df_user_8_14, start_date, 5)
CPB_7 = cost_per_buyer(df_ads_8_14, df_user_8_14, start_date, 7)
CPB_14 = cost_per_buyer(df_ads_8_14, df_user_8_14, start_date, 14)

print('CPB 1 = ', CPB_1,'\nCPB 3 = ', CPB_3,'\nCPB 5 = ', CPB_5,'\nCPB 7 = ', CPB_7,'\nCPB 14 = ', CPB_14)

# ROAS Доход (накопительный) / затраты на рекламу (накопительный) 
def ROAS(df_ads, df_user, start_date, n_day):   
    
    end_n_day = start_date + timedelta(days = n_day)

    df_user_n = df_user.loc[df_user['event_time'] < end_n_day.strftime('%Y-%m-%d')]
    revenue_n = df_user_n['event_revenue_usd'].sum()

    df_ads_n = df_ads.loc[df_ads['date'] < end_n_day.strftime('%Y-%m-%d')]
    cost_n = df_ads_n['spend'].sum()
    
    result_value = revenue_n / cost_n
    return result_value

ROAS_1 = ROAS(df_ads_8_14, df_user_8_14, start_date, 1)
ROAS_3 = ROAS(df_ads_8_14, df_user_8_14, start_date, 3)
ROAS_5 = ROAS(df_ads_8_14, df_user_8_14, start_date, 5)
ROAS_7 = ROAS(df_ads_8_14, df_user_8_14, start_date, 7)
ROAS_14 = ROAS(df_ads_8_14, df_user_8_14, start_date, 14)
ROAS_30 = ROAS(df_ads_8_14, df_user_8_14, start_date, 30)

print('ROAS 1 = ', ROAS_1,'\nROAS 3 = ', ROAS_3,'\nROAS 5 = ', ROAS_5,'\nROAS 7 = ', ROAS_7,'\nROAS 14 = ', ROAS_14,'\nROAS 30 = ', ROAS_30)

Cost =  1853283.0999999992
Impressions =  163672214
Installs =  268457
Buyers 3 =  2628
Buyers =  10883
CPM =  11.323138208419415
IPM =  1.6402112089716094
CPI =  6.903463496947366
ARPU_7 =  0.14497474925215664
ARPPU_7 =  9.47658472494929
ARPPU_7 =  9.47658472494929
CPB 1 =  110.04176363636363 
CPB 3 =  136.86626823194587 
CPB 5 =  160.55455982490267 
CPB 7 =  170.29156482587513 
CPB 14 =  170.29156482587513
ROAS 1 =  0.15474438941942206 
ROAS 3 =  0.16516128595719384 
ROAS 5 =  0.15873792204139253 
ROAS 7 =  0.1683401349594135 
ROAS 14 =  0.1683401349594135 
ROAS 30 =  0.1683401349594135
